## Run the STEMMUS_SCOPE model
Steps to run the STEMMUS_SCOPE model, including preprocessing and postprocessing, on Surf super computer Snellius or on a unix system.

In [1]:
import os
from pathlib import Path
import subprocess
import PyStemmusScope.iostreamer

#### Path to STEMMUS_SCOPE

STEMMUS_SCOPE executable file is located in the **private** repository on GitHub https://github.com/EcoExtreML/STEMMUS_SCOPE. You need to clone the repository locally and specify the path to it in the cell below. Make sure you have right access to the repository. 

In [2]:
# path to executable
path_to_model = Path("path_to_STEMMUS_SCOPE_repository")

#### Update/set config files

In [16]:
# the user must provide the correct path
# path to config file
path_to_config_file = path_to_model / "config_file_snellius.txt"
# Instantiate working directories handler from PyStemmusScope
iostreamer = PyStemmusScope.iostreamer.InputDir(path_to_config_file, path_to_model)

In [17]:
iostreamer.config

{'SoilPropertyPath': '/projects/0/einf2480/model_parameters/soil_property/',
 'InputPath': '/scratch-shared/ecoextreml/stemmus_scope/input/',
 'OutputPath': '/scratch-shared/ecoextreml/stemmus_scope/output/',
 'ForcingPath': '/projects/0/einf2480/forcing/plumber2_data/',
 'ForcingFileName': 'FI-Hyy_1996-2014_FLUXNET2015_Met.nc',
 'VegetationPropertyPath': '/projects/0/einf2480/model_parameters/vegetation_property/',
 'DurationSize': '17520'}

In [18]:
# edit config
iostreamer.config["NumberOfTimeSteps"] = "5"
iostreamer.config

{'SoilPropertyPath': '/projects/0/einf2480/model_parameters/soil_property/',
 'InputPath': '/scratch-shared/ecoextreml/stemmus_scope/input/',
 'OutputPath': '/scratch-shared/ecoextreml/stemmus_scope/output/',
 'ForcingPath': '/projects/0/einf2480/forcing/plumber2_data/',
 'ForcingFileName': 'FI-Hyy_1996-2014_FLUXNET2015_Met.nc',
 'VegetationPropertyPath': '/projects/0/einf2480/model_parameters/vegetation_property/',
 'DurationSize': '5'}

#### Create input directories, prepare input files 

In [10]:
# specify the forcing filenames
# forcing_filenames_list is by default none, if full_run is true, then all stations with forcing
# listed in the ForcingPath will be used.
forcing_filenames_list = ["NL-Hor_2008-2011_FLUXNET2015_Met.nc",
 "ZA-Kru_2000-2002_FLUXNET2015_Met.nc"]

work_dir_dict, config_path_dict = iostreamer.prepare_work_dir(forcing_filenames_list)

Prepare work directory /home/sarah/temp/ecoextreml/input/NL-Hor_20220519_1455 for the station: NL-Hor
Prepare work directory /home/sarah/temp/ecoextreml/input/ZA-Kru_20220519_1455 for the station: ZA-Kru


#### Run the model

If you run the model locally, you need to first intsall MATLAB Runtime, see instructions in the [readme](https://github.com/EcoExtreML/processing#readme). Then set `LD_LIBRARY_PATH`. To do this, uncomment the cell below and run it:

In [13]:
# # Set LD_LIBRARY_PATH
matlab_path = !whereis MATLAB
matlab_path = matlab_path.s.split(": ")[1]
os.environ['LD_LIBRARY_PATH'] = (
    f"{matlab_path}/MATLAB_Runtime/v910/runtime/glnxa64:"
    f"{matlab_path}/MATLAB_Runtime/v910/bin/glnxa64:"
    f"{matlab_path}/MATLAB_Runtime/v910/sys/os/glnxa64:"
    f"{matlab_path}/MATLAB_Runtime/v910/extern/bin/glnxa64:"
    f"{matlab_path}/MATLAB_Runtime/v910/sys/opengl/lib/glnxa64")
print(os.environ['LD_LIBRARY_PATH'])

/usr/local/MATLAB/MATLAB_Runtime/v910/runtime/glnxa64:/usr/local/MATLAB/MATLAB_Runtime/v910/bin/glnxa64:/usr/local/MATLAB/MATLAB_Runtime/v910/sys/os/glnxa64:/usr/local/MATLAB/MATLAB_Runtime/v910/extern/bin/glnxa64:/usr/local/MATLAB/MATLAB_Runtime/v910/sys/opengl/lib/glnxa64


In [14]:
# generate a text file
for ncfile in forcing_filenames_list:
    path_to_config = config_path_dict[ncfile]
    # set matlab log dir to slurm, otherwise java.log files are created in user home dir
    os.environ['MATLAB_LOG_DIR'] = str(work_dir_dict[ncfile])
    # run the model
    print(f"Run STEMMUS-SCOPE with the forcing: {ncfile}")
    result = subprocess.run([f"exe/STEMMUS_SCOPE {path_to_config}"], cwd = path_to_model, shell=True)
    result.check_returncode()

Run STEMMUS-SCOPE with the forcing: NL-Hor_2008-2011_FLUXNET2015_Met.nc
Reading config from /home/sarah/temp/ecoextreml/input/NL-Hor_20220519_1455/NL-Hor_20220519_1455_config.txt

 The calculations start now 
Run STEMMUS-SCOPE with the forcing: ZA-Kru_2000-2002_FLUXNET2015_Met.nc
Reading config from /home/sarah/temp/ecoextreml/input/ZA-Kru_20220519_1455/ZA-Kru_20220519_1455_config.txt

 The calculations start now 


#### Create output directories, prepare output files

In [15]:
# convert csv files to nc files
path_to_utils = Path(path_to_model, "utils/csv_to_nc")
for ncfile in forcing_filenames_list:
    path_to_config = config_path_dict[ncfile]
    result = subprocess.run(["python", Path(path_to_utils, "generate_netcdf_files.py"),
         "--config_file", path_to_config, "--variable_file",
         Path(path_to_utils, "Variables_will_be_in_NetCDF_file.csv")])
    result.check_returncode()

/home/sarah/temp/ecoextreml/output/NL-Hor_2022-05-19-1457/ECdata.csv
Reading variable metadata from /home/sarah/GitHub/STEMMUS_SCOPE/utils/csv_to_nc/Variables_will_be_in_NetCDF_file.csv
Creating /home/sarah/temp/ecoextreml/output/NL-Hor_2022-05-19-1457/NL-Hor_2022-05-19-1457_STEMMUS_SCOPE.nc 
Reading data from file: radiation.csv
Reading data from file: fluxes.csv
Reading data from file: surftemp.csv
Reading data from file: Sim_Temp.csv
Reading data from file: Sim_Theta.csv
Reading data from file: aerodyn.csv
Reading data from file: ECdata.csv
Done writing /home/sarah/temp/ecoextreml/output/NL-Hor_2022-05-19-1457/NL-Hor_2022-05-19-1457_STEMMUS_SCOPE.nc
/home/sarah/temp/ecoextreml/output/ZA-Kru_2022-05-19-1458/ECdata.csv
Reading variable metadata from /home/sarah/GitHub/STEMMUS_SCOPE/utils/csv_to_nc/Variables_will_be_in_NetCDF_file.csv
Creating /home/sarah/temp/ecoextreml/output/ZA-Kru_2022-05-19-1458/ZA-Kru_2022-05-19-1458_STEMMUS_SCOPE.nc 
Reading data from file: radiation.csv
Reading